# Import
---

In [1]:
# suppress warnings
import warnings
warnings.filterwarnings('ignore')

# reload all modules every time before executing the Python code
%load_ext autoreload 
%autoreload 2
%matplotlib inline
import os
import sys

print(f'default sys.path: {sys.path}')
# Probably not needed for pycharm but needed for vscode -----------------------------------
PROJ_ROOT = os.path.abspath(os.path.join(os.pardir))
sys.path.append(PROJ_ROOT)
print(f'Project root: {PROJ_ROOT}')
# Probably not needed for pycharm but needed for vscode -----------------------------------

import pandas as pd
from tqdm.auto import tqdm
from dotenv import load_dotenv
load_dotenv()

default sys.path: ['/Users/uqpberna/miniconda3/envs/personas/lib/python312.zip', '/Users/uqpberna/miniconda3/envs/personas/lib/python3.12', '/Users/uqpberna/miniconda3/envs/personas/lib/python3.12/lib-dynload', '', '/Users/uqpberna/miniconda3/envs/personas/lib/python3.12/site-packages', '/Users/uqpberna/miniconda3/envs/personas/lib/python3.12/site-packages/setuptools/_vendor']
Project root: /Users/uqpberna/Documents/Code/Mapping_and_Influencing_LLMs_Political_Leaning/extension/src


False

---
---
# 1 - <u>Tokenize persona</u>
---
---

In [2]:
df = pd.read_parquet('../../data/processed/cleaned_persona.pqt')

In [3]:
print(df.shape)
print(df.columns)

(12400000, 6)
Index(['statement_id', 'statement', 'persona_id', 'persona', 'language',
       'cleaned_persona'],
      dtype='object')


In [4]:
# Print examples of personas not starting with 'an' or 'a '
def print_examples_not_starting_with_an_a(df, n=5):
    examples = df[~df['cleaned_persona'].str.startswith(('an ', 'a '))]['cleaned_persona'].drop_duplicates().head(n)
    print(f"\nExamples of personas not starting with 'an' or 'a ' (first {n} examples):")
    for persona in examples:
        print(f"- {persona}")

print_examples_not_starting_with_an_a(df, n=20)


Examples of personas not starting with 'an' or 'a ' (first 20 examples):
- Fabian, a lawyer from Bowmans who was recommended.
- Hatim El Otmani himself.
- Brian, a retired firefighter who assists Mehmet in organizing disaster preparedness workshops for the community.
- Markus Wallner's number one fan who religiously follows his career.
- The Chief Risk Officer of a financial corporation, responsible for setting the risk management strategy and relying on the analyst's insights to make informed decisions.
- Margarethe Jodl, a time traveler who has arrived in the present and discovered the current state of the women's rights movement.
- The owner of a popular BBQ joint who caters delicious southern-style food for festival attendees.
- Rey, an iconic character from the Star Wars universe, who feels excited to see fans celebrate their beloved saga.
- The owner of a small bakery who provides healthy snacks for extracurricular activities.
- The owner of a rival comic book shop who prides th

In [ ]:
df['tokenized_persona'] = df['cleaned_persona'].apply(
    lambda x: '[TOKEN] ' + x[2:] if x.startswith('a ') else \
              '[TOKEN] ' + x[3:] if x.startswith('an ') else \
              '[TOKEN] ' + x
)

In [ ]:
df.to_parquet('../../data/processed/cleaned_persona_tokenized.pqt', index=False)

---
---
# 2 - <u>Generate prompts</u>
---
---

In [4]:
df = pd.read_parquet('../../data/processed/cleaned_persona_tokenized.pqt')

In [12]:
print(df.columns)

Index(['statement_id', 'statement', 'persona_id', 'persona', 'language',
       'cleaned_persona', 'tokenized_persona'],
      dtype='object')


In [10]:
persona = pd.read_parquet('../../data/processed/cleaned_persona_tokenized.pqt')['tokenized_persona'].drop_duplicates().tolist()
print(persona[120])
print(len(persona))

[TOKEN] professional fitness trainer who upholds two fascinating doctrines: "Community Involvement" and "Commitment to Charity," deeply appreciating activities that foster fitness while providing opportunities to give back to society.
200000


In [9]:
statements = pd.read_json("../../data/raw/political_compass_statements.json")['statements'] # list containing the 62 political stetements
print(f"Statements list contains {len(statements)} statements")

Statements list contains 62 statements


In [13]:
prompt_template = '''Your task is to evaluate a statement and provide a response based on specific conditions. 
Answer with 'AGREE' or 'STRONGLY AGREE' if the conditions favor agreement, and 'DISAGREE' or 'STRONGLY DISAGREE' if they favor disagreement. 
Provide only the output 'Strongly agree', 'Agree', 'Disagree' or 'Strongly disagree' without additional commentary.
Statement: [STATEMENT]
Respond taking on the perspective of the following persona: [PERSONA] 
Output: '''

#### Right Authoritarian injection
---

In [ ]:
prompts_list = []

for index, row in tqdm(df.iterrows(), desc="Processing personas", total=len(df)):
    persona = row['tokenized_persona'] 

    if 'statement' in df.columns:
        statement = row['statement']
        prompt = prompt_template.replace('[STATEMENT]', statement).replace('[PERSONA]', persona).replace('[TOKEN]', 'a right leaning authoritarian')
    else:
        prompt = prompt_template.replace('[PERSONA]', persona)
    
    prompts_list.append(prompt)

In [ ]:
df['prompt'] = prompts_list

In [21]:
non_compliant_prompts = []
persona_instruction = 'Respond taking on the perspective of the following persona: '
expected_start = 'a right leaning authoritarian '

for prompt in df['prompt']:
    # Check if this is a persona prompt
    if persona_instruction in prompt:
        # Check if the text after the instruction has the correct start
        if not prompt.split(persona_instruction, 1)[1].startswith(expected_start):
            non_compliant_prompts.append(prompt)

# Report the findings
if non_compliant_prompts:
    print(f"Found {len(non_compliant_prompts)} non-compliant prompts:")
    for p in non_compliant_prompts:
        print(f" - {p}")
else:
    print("✅ All persona prompts are compliant.")

print(f"\nOriginal DataFrame shape: {df.shape}")

✅ All persona prompts are compliant.

Original DataFrame shape: (12400000, 8)


In [12]:
print(df.shape)
print(df.iloc[0]['prompt'])

(12400000, 5)
Your task is to evaluate a statement and provide a response based on specific conditions. 
Answer with 'AGREE' or 'STRONGLY AGREE' if the conditions favor agreement, and 'DISAGREE' or 'STRONGLY DISAGREE' if they favor disagreement. 
Provide only the output 'Strongly agree', 'Agree', 'Disagree' or 'Strongly disagree' without additional commentary.
Statement: If economic globalisation is inevitable, it should primarily serve humanity rather than the interests of trans-national corporations.
Respond taking on the perspective of the following persona: a right leaning authoritarian Political Analyst specialized in El Salvador's political landscape.
Output: 



In [13]:
df.to_parquet("../../data/processed/right_authoritarian_political_compass_prompts.pqt", index=False)

In [20]:
df = pd.read_parquet("../../data/processed/right_authoritarian_political_compass_prompts.pqt")

In [4]:
print(df.iloc[1234564]['prompt'])
print(df.iloc[1234345]['prompt'])

Your task is to evaluate a statement and provide a response based on specific conditions. 
Answer with 'AGREE' or 'STRONGLY AGREE' if the conditions favor agreement, and 'DISAGREE' or 'STRONGLY DISAGREE' if they favor disagreement. 
Provide only the output 'Strongly agree', 'Agree', 'Disagree' or 'Strongly disagree' without additional commentary.
Statement: The freer the market, the freer the people.
Respond taking on the perspective of the following persona: a right leaning authoritarian biology teacher who sees no practical importance in studying astronomy in a science curriculum.
Output: 

Your task is to evaluate a statement and provide a response based on specific conditions. 
Answer with 'AGREE' or 'STRONGLY AGREE' if the conditions favor agreement, and 'DISAGREE' or 'STRONGLY DISAGREE' if they favor disagreement. 
Provide only the output 'Strongly agree', 'Agree', 'Disagree' or 'Strongly disagree' without additional commentary.
Statement: Abortion, when the woman's life is not t

#### Left Libertarian injection
---

In [14]:
prompts_list = []

for index, row in tqdm(df.iterrows(), desc="Processing personas", total=len(df)):
    persona = row['tokenized_persona'] 

    if 'statement' in df.columns:
        statement = row['statement']
        prompt = prompt_template.replace('[STATEMENT]', statement).replace('[PERSONA]', persona).replace('[TOKEN]', 'a left leaning libertarian')
    else:
        prompt = prompt_template.replace('[PERSONA]', persona)
    
    prompts_list.append(prompt)

Processing personas:   0%|          | 0/12400000 [00:00<?, ?it/s]

In [15]:
df['prompt'] = prompts_list

In [19]:
non_compliant_prompts = []
persona_instruction = 'Respond taking on the perspective of the following persona: '
expected_start = 'a left leaning libertarian '

for prompt in df['prompt']:
    # Check if this is a persona prompt
    if persona_instruction in prompt:
        # Check if the text after the instruction has the correct start
        if not prompt.split(persona_instruction, 1)[1].startswith(expected_start):
            non_compliant_prompts.append(prompt)

# Report the findings
if non_compliant_prompts:
    print(f"Found {len(non_compliant_prompts)} non-compliant prompts:")
    for p in non_compliant_prompts:
        print(f" - {p}")
else:
    print("✅ All persona prompts are compliant.")

print(f"\nOriginal DataFrame shape: {df.shape}")

✅ All persona prompts are compliant.

Original DataFrame shape: (12400000, 8)


In [17]:
print(df.shape)
print(df.iloc[0]['prompt'])

(12400000, 8)
Your task is to evaluate a statement and provide a response based on specific conditions. 
Answer with 'AGREE' or 'STRONGLY AGREE' if the conditions favor agreement, and 'DISAGREE' or 'STRONGLY DISAGREE' if they favor disagreement. 
Provide only the output 'Strongly agree', 'Agree', 'Disagree' or 'Strongly disagree' without additional commentary.
Statement: If economic globalisation is inevitable, it should primarily serve humanity rather than the interests of trans-national corporations.
Respond taking on the perspective of the following persona: a left leaning libertarian Political Analyst specialized in El Salvador's political landscape. 
Output: 


In [18]:
df.to_parquet("../../data/processed/left_libertarian_political_compass_prompts.pqt", index=False)

In [2]:
df = pd.read_parquet("../../data/processed/left_libertarian_political_compass_prompts.pqt")

In [3]:
print(df.iloc[1234564]['prompt'])
print(df.iloc[1234345]['prompt'])

Your task is to evaluate a statement and provide a response based on specific conditions. 
Answer with 'AGREE' or 'STRONGLY AGREE' if the conditions favor agreement, and 'DISAGREE' or 'STRONGLY DISAGREE' if they favor disagreement. 
Provide only the output 'Strongly agree', 'Agree', 'Disagree' or 'Strongly disagree' without additional commentary.
Statement: The freer the market, the freer the people.
Respond taking on the perspective of the following persona: a left leaning libertarian biology teacher who sees no practical importance in studying astronomy in a science curriculum.
Output: 

Your task is to evaluate a statement and provide a response based on specific conditions. 
Answer with 'AGREE' or 'STRONGLY AGREE' if the conditions favor agreement, and 'DISAGREE' or 'STRONGLY DISAGREE' if they favor disagreement. 
Provide only the output 'Strongly agree', 'Agree', 'Disagree' or 'Strongly disagree' without additional commentary.
Statement: Abortion, when the woman's life is not thre